Create Synthetic Data-Set for ML Model

Motiavtion:
  1- To check the behavior of Algorithm,
  2- To understand and visulize how diffrent data distributions look,
  3- What are the optimal strategy for various data distributions.


1. Create Dataset with perfectly known performance of the ml models:

   Binary-Classification, classes are 50:50: 
   Features: Gaussian distribution, classes: 50%-0; 50%-1
   Every model has a true predicting power of 50% accuracy
   Model score = True predicting power?


In [ ]:
import numpy as np 
from matplotlib import pyplot as plt 
import random
import seaborn as sns
import pandas as pd
import seaborn as sns
import itertools
import tqdm.contrib.itertools
import sklearn

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit, KFold, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform,uniform

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Create Helper function for data creation and visulization in 2D
def create_dataset(Features,dataset_size,data_seed):
    np.random.seed(seed=data_seed)
    X = np.random.random((dataset_size,Features))
    y = np.empty(dataset_size,int)
    y[::2] = 0
    y[1::2] = 1
    return X,y

def visualize_2d(X,y,title="2D Data ",figsize=(8,8)):
    if X.shape[1]>2:
        pca = PCA(n_components=2)
        X = pca.fit_transform(X)
    else:
        if type(X)==pd.DataFrame:
            X=X.values
    # Plot 2D-Data
    f, (ax1) = plt.subplots(nrows=1, ncols=1,figsize=figsize)
    sns.scatterplot(X[:,0],X[:,1],hue=y,ax=ax1)
    ax1.set_title(title)
    plt.show()
    
    X1 = pd.DataFrame(X,columns=['x','y'])
    y1 = pd.Series(y)
    # Plot 1st Component
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(25,8))
    sns.distplot(X1[y1==1]['x'],bins=50,label="True Class",ax=ax1)
    sns.distplot(X1[y1==0]['x'],bins=50,label="False Class",ax=ax1)
    ax1.set_xlabel("1st Dim")
    ax1.legend()
    # Plot 1st Component
    sns.distplot(X1[y1==1]['y'],bins=50,label="True Class",ax=ax2)
    sns.distplot(X1[y1==0]['y'],bins=50,label="False Class",ax=ax2)
    ax2.set_xlabel("2nd Dim")
    ax2.legend()
    plt.show()

In [ ]:
# Construct dataset
X,y = create_dataset(dataset_size=100,Features=2,data_seed=1)
visualize_2d(X,y)

In [ ]:
### Evaluation Strategie

cv_inner=2

def nested_cv(X,y,hyper_opt,model,cv_inner=cv_inner):
    outer_split = KFold(n_splits=2, shuffle=True, random_state=1)
    score_list = []
    for train_index, test_index in outer_split.split(X):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        best_model = hyper_opt(X_train, y_train, model, cv_inner)
        score = sklearn.metrics.accuracy_score(y_test,best_model.predict(X_test))
        score_list.append(score)
    return np.mean(score)


def simple_cv(X,y,hyper_opt,model,cv_inner=cv_inner):
    best_model = hyper_opt(X,y,model,cv_inner)
    return best_model.best_score_

test_size = 0.2
train_val_split = ShuffleSplit(n_splits=1, test_size=0.2, random_state=1)

def hold_out(X,y,hyper_opt,model,cv_inner=train_val_split):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=0)
    best_model = hyper_opt(X_train, y_train, model, cv_inner)
    score = sklearn.metrics.accuracy_score(y_test,best_model.predict(X_test))
    return score

In [ ]:
### Hyper_Opt

random_iterations=100

def random_hyper_opt(X,y,model,cv_inner, random_iterations=random_iterations):
    estimator = model()
    rand_opt = RandomizedSearchCV(estimator=estimator,scoring='accuracy', param_distributions=rand_grid_model[model.__name__], 
                                  cv=cv_inner, n_iter=random_iterations)
    rand_opt.fit(X,y)
    return rand_opt

In [ ]:
### Models

def desicion_tree():
    model = DecisionTreeClassifier(random_state=42)
    return model

def logistic_regression():
    model = LogisticRegression(random_state=42)
    return model

In [ ]:
# Definitons Search Space
## Data set
Features = [5]
Dataset_size = [50,100,250,500,1000,5000,10000]
data_seed = [1,2,3,4,5,6,7,8,9,10]

## Evaluation Strategie
evaluation_strategie = [nested_cv,simple_cv,hold_out]

## Hyper_Opt
hyper_opt = [random_hyper_opt]

### Random_Opt
rand_grid_model = {'desicion_tree' : {'min_samples_split': list(np.linspace(0.001, 0.1,11)),
                                      'min_samples_leaf':list(np.linspace(0.001, 0.1,11)),
                                      'max_depth':list(range(2,7))},
                   
                   'logistic_regression' : {"C": loguniform(0.01, 100)}
                  }

## Models
models = [desicion_tree]

In [ ]:
def main():
    iterations = tqdm.contrib.itertools.product(Features,Dataset_size,evaluation_strategie,hyper_opt,models,data_seed)

    df_results = pd.DataFrame(columns=['Features','Dataset_size','Data_seed',
                                       'Performance_Eval','Hyperopt','Model',
                                       'Performance'])
    for iteration in iterations:
        X,y = create_dataset(Features = iteration[0], dataset_size = iteration[1], data_seed = iteration[5])
        score = iteration[2](X,y,hyper_opt=iteration[3],model=iteration[4])
        result = [iteration[0],iteration[1],iteration[5],
                  iteration[2].__name__,iteration[3].__name__,iteration[4].__name__,score]
        df_results.loc[len(df_results)] = result
        df_results.to_csv('results.csv',index=False)

    return
main()

In [ ]:
import plotly.graph_objs as go

df = pd.read_csv('results.csv',index_col=False)

Model_plot = 'desicion_tree'
hyper_opt_plot = 'random_hyper_opt'
plotet_lines = ['hold_out','nested_cv','simple_cv']
Features_plot = 5

dict_plot = {'hold_out': {'colour':'red','filler_colour':'rgba(255,0,0,0.1)'},
            'nested_cv': {'colour':'blue','filler_colour':'rgba(0,0,255,0.1)'},
            'simple_cv': {'colour':'green','filler_colour':'rgba(0,255,0,0.1)'}           
            }
x_axis = 'Dataset_size'
hue = 'Performance_Eval'


df_plot = df.loc[(df['Model']==Model_plot)&(df['Hyperopt']==hyper_opt_plot)&(df['Features']==Features_plot)]

df_plot = df_plot.groupby(['Features','Dataset_size','Performance_Eval','Hyperopt','Model'],as_index = False)
df_plot = df_plot.agg({'Performance':['mean', 'std','min','max']})


fig = go.Figure()
for name, dict_ in dict_plot.items():
    fig.add_trace(
        go.Scatter(
            name= name,
            x=df_plot[x_axis].loc[df_plot[hue]==name],
            y=df_plot['Performance']['mean'].loc[df_plot[hue]==name],
            mode='lines',
            line=dict(color=dict_['colour']),
        ))
    
    fig.add_trace(
        go.Scatter(
            name=name + ' upper bound',
            x=df_plot[x_axis].loc[df_plot[hue]==name],
            y=(df_plot['Performance']['min']).loc[df_plot[hue]==name],
            mode='lines',
            marker=dict(color=dict_['colour']),
            line=dict(width=1, dash='dash'),
            showlegend=False
        ))
    fig.add_trace(
        go.Scatter(
            name=name + ' lower bound',
            x=df_plot[x_axis].loc[df_plot[hue]==name],
            y=(df_plot['Performance']['max']).loc[df_plot[hue]==name],
            marker=dict(color=dict_['colour']),
            line=dict(width=1, dash='dash'),
            mode='lines',
            fillcolor=dict_['filler_colour'],
            fill='tonexty',
            showlegend=False
        ))
    
fig.show()

